In [16]:
%%capture

!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y
!pip install rasterio
!pip install fiona
!pip install geopandas
!pip install PyCRS
!pip install -i https://test.pypi.org/simple/ gis-utils-pkg-dillhicks==0.0.4

from rasterio.features import shapes
import os
from os.path import join
import rasterio
import geopandas as gpd
from gis_utils import raster
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [42]:
def get_files():
    labels = []
    #getting labeled tif files in the ML Label Paint Directory
    for root, dirs, files in os.walk("/content/drive/Shared drives/SIO and E4E Mangroves /Data/Labelled Data/ML Paint Labels" ):
        for file in files: 
            if file.endswith("_MLPaintLabels.tif"):
                labels.append([root, file,
                            file.replace("_MLPaintLabels.tif","")])
    cur = []

    #Getting template tif file and 
    for label in labels:
        for root, dirs, files in os.walk("/content/drive/Shared drives/SIO and E4E Mangroves /Data/Orthomosaics/"):
            for file in files:
                #finding all normal tifs
                if file.startswith(label[2] + ".tif"):
                    dest_shp = join(label[0], os.path.splitext(label[1])[0] + ".shp")
                    file_list = [join(label[0],label[1]),
                                 dest_shp,
                                 join(root,file)]
                                 print
                    if not(file_list in cur) and not(os.path.isfile(dest_shp)):
                            cur.append(file_list)
    return cur

In [52]:

def polygonize(label_path, output_file="None", template_path="None", georeferenced = True):
    
    src, _=  raster.load_image(label_path) 
    if (template_path != "None") and (georeferenced == False):
        print("Input label is not georeferenced, since we have a  is still good.")
        orig, _ = raster.load_image(template_path)
        array = src.read()
        results = ({'properties': {'raster_val': v}, 'geometry': s}for i, (s, v) in enumerate(shapes(array, mask=None, transform=orig.transform)))
        geoms = list(results)
        gdf_polygonized_raster  = gpd.GeoDataFrame.from_features(geoms)


    else:
        array = src.read()
        results = ({'properties': {'raster_val': v}, 'geometry': s}for i, (s, v) in enumerate(shapes(array, mask=None, transform=src.transform)))
        geoms = list(results)
        gdf_polygonized_raster  = gpd.GeoDataFrame.from_features(geoms)

    
    if output_file != "None":
        gdf_polygonized_raster.to_file(output_file)

    return gdf_polygonized_raster




In [57]:
cur = get_files()

for file_list in cur:
    polygonize(file_list[0], file_list[1], template_path = file_list[2], georeferenced = False)

/usr/local/lib/python3.6/dist-packages/rasterio/__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
